## GCN CODE

In [ ]:
!pip -q install rdkit
!pip -q install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-2.1.0+cu121.html

In [ ]:
from rdkit import Chem
from rdkit.Chem import rdchem, rdmolops, Descriptors, rdMolDescriptors
import pandas as pd
import numpy as np
from rdkit import Chem
import pandas as pd
from rdkit import Chem
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch_geometric.data import Data
import torch
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.loader import DataLoader
import torch.nn.functional as F

def atom_features(atom):
    """Create an atom feature vector."""
    return [
        atom.GetAtomicNum(),
        atom.GetDegree(),
        atom.GetFormalCharge(),
        atom.GetHybridization().real,
        atom.GetIsAromatic(),
        # Add more atom-level features here.
    ]

def bond_features(bond):
    """Create a bond feature vector."""
    bt = bond.GetBondType()
    return [
        bt == rdchem.BondType.SINGLE,
        bt == rdchem.BondType.DOUBLE,
        bt == rdchem.BondType.TRIPLE,
        bt == rdchem.BondType.AROMATIC,
        bond.GetIsConjugated(),
        bond.IsInRing(),
        # Add more bond-level features here.
    ]

def smiles_to_graph(smiles):
    """
    Convert a SMILES string to a graph with nodes as atoms and edges as bonds,
    including additional features for atoms and bonds.
    """
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)  # Add hydrogens to the molecule.

    # Get atom features
    atom_features_list = [atom_features(atom) for atom in mol.GetAtoms()]

    # Get bond features (as a matrix)
    num_atoms = mol.GetNumAtoms()
    bond_features_matrix = np.zeros((num_atoms, num_atoms, len(bond_features(mol.GetBonds()[0]))))

    for bond in mol.GetBonds():
        idx = bond.GetBeginAtomIdx()
        jdx = bond.GetEndAtomIdx()
        bond_feat = bond_features(bond)

        bond_features_matrix[idx, jdx, :] = bond_feat
        bond_features_matrix[jdx, idx, :] = bond_feat  # Bond is undirected

    # Create adjacency matrix where entries are 1 for bonded atom pairs
    adjacency_matrix = rdmolops.GetAdjacencyMatrix(mol)

    graph = {
        "atom_features": atom_features_list,
        "bond_features": bond_features_matrix,
        "adjacency_matrix": adjacency_matrix
        # Add more graph-level features if needed
    }

    return graph


In [ ]:
def read_and_process_dataset(file_path, target_columns):
    df = pd.read_csv(file_path)

    smiles_column = 'Smiles'
    graphs = []
    labels = []

    for _, row in df.iterrows():
        graph = smiles_to_graph(row[smiles_column])
        labels.append([row[col] for col in target_columns])
        graphs.append(graph)

    return graphs, np.array(labels)

# Replace with the path to your  dataset and target columns
file_path = '/content/LCIA_DATASET_CLEANED_V3.csv'
target_columns = ['HTP']  # Replace with target column names

graphs, labels = read_and_process_dataset(file_path, target_columns)

# Standardize target columns
scaler = StandardScaler()
labels_scaled = scaler.fit_transform(labels)

# Convert graphs to PyG data objects
pyg_graphs = [
    Data(
        x=torch.tensor(graph['atom_features'], dtype=torch.float),
        edge_index=torch.tensor(np.nonzero(graph['adjacency_matrix'])).type(torch.long),
        y=torch.tensor([labels_scaled[i]], dtype=torch.float) # GWP value for each graph
      )
for i, graph in enumerate(graphs)
]


# Split the data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(pyg_graphs, labels_scaled, train_size=0.6, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, train_size=0.5, random_state=42)



In [ ]:
num_node_features = pyg_graphs[0].num_node_features
#num_classes = labels_scaled.shape[1]  # This is the number of target columns

In [ ]:

class GCN(torch.nn.Module):
    def __init__(self, num_node_features):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, 64)
        self.conv2 = GCNConv(64, 32)
        self.fc = torch.nn.Linear(32, 1)  # Output one value per graph

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)

        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)

        x = global_mean_pool(x, batch)  # Pool node features to get graph-level features
        x = self.fc(x)  # Output one value per graph

        return x

# Instantiate the model
model = GCN(num_node_features)



In [ ]:


# Convert the lists of PyG Data objects into DataLoader objects
train_loader = DataLoader(X_train, batch_size=8, shuffle=True)
val_loader = DataLoader(X_val, batch_size=8, shuffle=False)
test_loader = DataLoader(X_test, batch_size=8, shuffle=False)

# Instantiate the model
model = GCN(num_node_features)

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Define the loss function
loss_func = torch.nn.MSELoss()

# Define the learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.8, patience=3, verbose=True)

# Early stopping parameters
early_stopping_patience = 25
best_val_loss = float('inf')
epochs_no_improve = 0


In [ ]:
from torch_geometric.loader import DataLoader

def train(model, train_loader, val_loader, optimizer, criterion, scheduler, epochs, early_stopping_patience):
    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for data in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)

        # Validation step
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for data in val_loader:
                output = model(data)
                loss = criterion(output, data.y)
                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_loader)

        # Scheduler step (for learning rate decay)
        scheduler.step(avg_val_loss)

        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')

        # Early stopping check
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve == early_stopping_patience:
                print(f'Early stopping triggered after {epoch+1} epochs!')
                break

    return model

# Number of epochs
epochs = 300

# Early stopping patience
early_stopping_patience = 25

# Train the model
trained_model = train(model, train_loader, val_loader, optimizer, loss_func, scheduler, epochs, early_stopping_patience)


Epoch 1/300, Train Loss: 1.9190, Val Loss: 0.0187
Epoch 2/300, Train Loss: 1.7026, Val Loss: 0.0334
Epoch 3/300, Train Loss: 1.7240, Val Loss: 0.0241
Epoch 4/300, Train Loss: 1.7723, Val Loss: 0.0284
Epoch 00005: reducing learning rate of group 0 to 8.0000e-04.
Epoch 5/300, Train Loss: 1.7499, Val Loss: 0.0269
Epoch 6/300, Train Loss: 1.7191, Val Loss: 0.0493
Epoch 7/300, Train Loss: 1.7216, Val Loss: 0.0666
Epoch 8/300, Train Loss: 1.7282, Val Loss: 0.0214
Epoch 9/300, Train Loss: 1.7228, Val Loss: 0.0127
Epoch 10/300, Train Loss: 1.7075, Val Loss: 0.0186
Epoch 11/300, Train Loss: 1.6676, Val Loss: 0.0197
Epoch 12/300, Train Loss: 1.6871, Val Loss: 0.0177
Epoch 00013: reducing learning rate of group 0 to 6.4000e-04.
Epoch 13/300, Train Loss: 1.7049, Val Loss: 0.0209
Epoch 14/300, Train Loss: 1.6853, Val Loss: 0.0245
Epoch 15/300, Train Loss: 1.6842, Val Loss: 0.0250
Epoch 16/300, Train Loss: 1.6665, Val Loss: 0.0299
Epoch 00017: reducing learning rate of group 0 to 5.1200e-04.
Epoch 1

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def evaluate_model(model, data_loader):
    model.eval()
    predictions, actuals = [], []
    with torch.no_grad():
        for data in data_loader:
            output = model(data)
            predictions.append(output.numpy())
            actuals.append(data.y.numpy())

    predictions = np.vstack(predictions)
    actuals = np.vstack(actuals)

    mse = mean_squared_error(actuals, predictions)
    mae = mean_absolute_error(actuals, predictions)
    r2 = r2_score(actuals, predictions)

    return mae, mse, r2


In [ ]:
# Evaluate on the validation set
val_mae, val_mse, val_r2 = evaluate_model(trained_model, val_loader)
print(f'Validation MAE: {val_mae}')
print(f'Validation MSE: {val_mse}')
print(f'Validation R²: {val_r2}')

# Evaluate on the test set
test_mae, test_mse, test_r2 = evaluate_model(trained_model, test_loader)
print(f'Test MAE: {test_mae}')
print(f'Test MSE: {test_mse}')
print(f'Test R²: {test_r2}')

Validation MAE: 0.13007131218910217
Validation MSE: 0.02008882164955139
Validation R²: -7.2505609208639985
Test MAE: 0.12080761045217514
Test MSE: 0.017602864652872086
Test R²: -5.423544480582923


In [ ]:


# # Check the shapes of the tensors
# for i, data in enumerate(pyg_graphs[:5]):  # Check the first 5 graphs as an example
#     print(f"Graph {i}:")
#     print(f"Number of nodes: {data.num_nodes}")
#     print(f"Shape of node features (x): {data.x.shape}")
#     print(f"Shape of edge_index: {data.edge_index.shape}")
#     print("Sample node features:", data.x[:5])  # Print the features of the first 5 nodes
#     print("Sample edges:", data.edge_index[:, :5])  # Print the first 5 edges
#     print("\n")

#check against a known molecule
# known_smiles = "CCO"  # Example SMILES for ethanol
# known_graph = smiles_to_graph(known_smiles)
# ethanol_data = Data(
#     x=torch.tensor(np.array(known_graph['atom_features']), dtype=torch.float),
#     edge_index=torch.tensor(np.nonzero(known_graph['adjacency_matrix'])).type(torch.long)
# )

# print("Known molecule - Ethanol:")
# print("Number of nodes:", ethanol_data.num_nodes)
# print("Shape of node features (x):", ethanol_data.x.shape)
# print("Shape of edge_index:", ethanol_data.edge_index.shape)
# print("Node features:", ethanol_data.x)
# print("Edges:", ethanol_data.edge_index)


## Ensamble method

In [ ]:
import numpy as np
import torch
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from torch_geometric.loader import DataLoader

num_models = 50
val_metrics = {'mae': [], 'mse': [], 'r2': []}
test_metrics = {'mae': [], 'mse': [], 'r2': []}

for i in range(num_models):
    # Split the dataset differently each time
    X_train, X_temp, y_train, y_temp = train_test_split(pyg_graphs, labels_scaled, train_size=0.8, random_state=i)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, train_size=0.5, random_state=i)

    # Create DataLoaders for each split
    train_loader = DataLoader(X_train, batch_size=8, shuffle=True)
    val_loader = DataLoader(X_val, batch_size=8, shuffle=False)
    test_loader = DataLoader(X_test, batch_size=8, shuffle=False)

    # Instantiate and train the model
    model = GCN(num_node_features)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_func = torch.nn.MSELoss()
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.8, patience=3, verbose=True)

    trained_model = train(model, train_loader, val_loader, optimizer, loss_func, scheduler, epochs, early_stopping_patience)

    # Evaluate the model
    val_mae, val_mse, val_r2 = evaluate_model(trained_model, val_loader)
    test_mae, test_mse, test_r2 = evaluate_model(trained_model, test_loader)

    # Save the metrics
    val_metrics['mae'].append(val_mae)
    val_metrics['mse'].append(val_mse)
    val_metrics['r2'].append(val_r2)

    test_metrics['mae'].append(test_mae)
    test_metrics['mse'].append(test_mse)
    test_metrics['r2'].append(test_r2)

# Calculate average of the metrics
avg_val_metrics = {metric: np.mean(values) for metric, values in val_metrics.items()}
avg_test_metrics = {metric: np.mean(values) for metric, values in test_metrics.items()}

# Prepare data for DataFrame
data = {
    'MAE Validation': [avg_val_metrics['mae']],
    'MSE Validation': [avg_val_metrics['mse']],
    'R² Validation': [avg_val_metrics['r2']],
    'MAE Test': [avg_test_metrics['mae']],
    'MSE Test': [avg_test_metrics['mse']],
    'R² Test': [avg_test_metrics['r2']]
}

# Create DataFrame
results_df = pd.DataFrame(data)

# Save the DataFrame to a CSV file in the current directory
csv_file_path = 'model_metrics.csv'
results_df.to_csv(csv_file_path, index=False)

# Print out the path to the CSV file
print(f'Metrics saved to {csv_file_path}')



Epoch 1/300, Train Loss: 1.4457, Val Loss: 0.0109
Epoch 2/300, Train Loss: 1.2838, Val Loss: 0.0015
Epoch 3/300, Train Loss: 1.2612, Val Loss: 0.0017
Epoch 4/300, Train Loss: 1.2745, Val Loss: 0.0026
Epoch 5/300, Train Loss: 1.2873, Val Loss: 0.0025
Epoch 00006: reducing learning rate of group 0 to 8.0000e-04.
Epoch 6/300, Train Loss: 1.2496, Val Loss: 0.0032
Epoch 7/300, Train Loss: 1.2371, Val Loss: 0.0035
Epoch 8/300, Train Loss: 1.2445, Val Loss: 0.0080
Epoch 9/300, Train Loss: 1.2608, Val Loss: 0.0050
Epoch 00010: reducing learning rate of group 0 to 6.4000e-04.
Epoch 10/300, Train Loss: 1.2177, Val Loss: 0.0055
Epoch 11/300, Train Loss: 1.2722, Val Loss: 0.0085
Epoch 12/300, Train Loss: 1.2286, Val Loss: 0.0096
Epoch 13/300, Train Loss: 1.2452, Val Loss: 0.0109
Epoch 00014: reducing learning rate of group 0 to 5.1200e-04.
Epoch 14/300, Train Loss: 1.2067, Val Loss: 0.0093
Epoch 15/300, Train Loss: 1.2260, Val Loss: 0.0096
Epoch 16/300, Train Loss: 1.2145, Val Loss: 0.0098
Epoch 1

In [ ]:
# import pandas as pd
# import numpy as np
# import random
# import torch
# from rdkit import Chem
# from rdkit.Chem import rdchem, rdmolops
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from torch_geometric.data import Data
# from torch_geometric.nn import GCNConv, global_mean_pool
# from torch_geometric.loader import DataLoader
# from torch.optim.lr_scheduler import ReduceLROnPlateau
# import torch.nn.functional as F
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# # Define your atom_features, bond_features, smiles_to_graph functions here
# def atom_features(atom):
#     """Create an atom feature vector."""
#     return [
#         atom.GetAtomicNum(),
#         atom.GetDegree(),
#         atom.GetFormalCharge(),
#         atom.GetHybridization().real,
#         atom.GetIsAromatic(),
#         # Add more atom-level features here.
#     ]

# def bond_features(bond):
#     """Create a bond feature vector."""
#     bt = bond.GetBondType()
#     return [
#         bt == rdchem.BondType.SINGLE,
#         bt == rdchem.BondType.DOUBLE,
#         bt == rdchem.BondType.TRIPLE,
#         bt == rdchem.BondType.AROMATIC,
#         bond.GetIsConjugated(),
#         bond.IsInRing(),
#         # Add more bond-level features here.
#     ]

# def smiles_to_graph(smiles):
#     """
#     Convert a SMILES string to a graph with nodes as atoms and edges as bonds,
#     including additional features for atoms and bonds.
#     """
#     mol = Chem.MolFromSmiles(smiles)
#     mol = Chem.AddHs(mol)  # Add hydrogens to the molecule.

#     # Get atom features
#     atom_features_list = [atom_features(atom) for atom in mol.GetAtoms()]

#     # Get bond features (as a matrix)
#     num_atoms = mol.GetNumAtoms()
#     bond_features_matrix = np.zeros((num_atoms, num_atoms, len(bond_features(mol.GetBonds()[0]))))

#     for bond in mol.GetBonds():
#         idx = bond.GetBeginAtomIdx()
#         jdx = bond.GetEndAtomIdx()
#         bond_feat = bond_features(bond)

#         bond_features_matrix[idx, jdx, :] = bond_feat
#         bond_features_matrix[jdx, idx, :] = bond_feat  # Bond is undirected

#     # Create adjacency matrix where entries are 1 for bonded atom pairs
#     adjacency_matrix = rdmolops.GetAdjacencyMatrix(mol)

#     graph = {
#         "atom_features": atom_features_list,
#         "bond_features": bond_features_matrix,
#         "adjacency_matrix": adjacency_matrix
#         # Add more graph-level features if needed
#     }

#     return graph

# def read_and_process_dataset(file_path, target_columns):
#     df = pd.read_csv(file_path)

#     smiles_column = 'Smiles'
#     graphs = []
#     labels = []

#     for _, row in df.iterrows():
#         graph = smiles_to_graph(row[smiles_column])
#         labels.append([row[col] for col in target_columns])
#         graphs.append(graph)

#     return graphs, np.array(labels)

# # GCN Model Definition
# class GCN(torch.nn.Module):
#     def __init__(self, num_node_features):
#         super(GCN, self).__init__()
#         self.conv1 = GCNConv(num_node_features, 64)
#         self.conv2 = GCNConv(64, 32)
#         self.fc = torch.nn.Linear(32, 1)  # Output one value per graph

#     def forward(self, data):
#         x, edge_index, batch = data.x, data.edge_index, data.batch

#         x = self.conv1(x, edge_index)
#         x = torch.relu(x)
#         x = F.dropout(x, p=0.5, training=self.training)

#         x = self.conv2(x, edge_index)
#         x = torch.relu(x)
#         x = F.dropout(x, p=0.5, training=self.training)

#         x = global_mean_pool(x, batch)  # Pool node features to get graph-level features
#         x = self.fc(x)  # Output one value per graph

#         return x

# # Training Function
# def train(model, train_loader, val_loader, optimizer, criterion, scheduler, epochs, early_stopping_patience):
#     best_val_loss = float('inf')
#     epochs_no_improve = 0

#     for epoch in range(epochs):
#         model.train()
#         total_loss = 0
#         for data in train_loader:
#             optimizer.zero_grad()
#             output = model(data)
#             loss = criterion(output, data.y)
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()

#         avg_train_loss = total_loss / len(train_loader)

#         # Validation step
#         model.eval()
#         total_val_loss = 0
#         with torch.no_grad():
#             for data in val_loader:
#                 output = model(data)
#                 loss = criterion(output, data.y)
#                 total_val_loss += loss.item()

#         avg_val_loss = total_val_loss / len(val_loader)

#         # Scheduler step (for learning rate decay)
#         scheduler.step(avg_val_loss)

#         print(f'Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')

#         # Early stopping check
#         if avg_val_loss < best_val_loss:
#             best_val_loss = avg_val_loss
#             epochs_no_improve = 0
#         else:
#             epochs_no_improve += 1
#             if epochs_no_improve == early_stopping_patience:
#                 print(f'Early stopping triggered after {epoch+1} epochs!')
#                 break

#     return model

# # Evaluation Function
# def evaluate_model(model, data_loader):
#     model.eval()
#     predictions, actuals = [], []
#     with torch.no_grad():
#         for data in data_loader:
#             output = model(data)
#             predictions.append(output.numpy())
#             actuals.append(data.y.numpy())

#     predictions = np.vstack(predictions)
#     actuals = np.vstack(actuals)

#     mse = mean_squared_error(actuals, predictions)
#     mae = mean_absolute_error(actuals, predictions)
#     r2 = r2_score(actuals, predictions)

#     return mae, mse, r2

# # Ensemble method across all targets
# file_path = '/content/LCIA_DATASET_CLEANED_V3.csv'
# target_columns = ["GWP", "HTP", "MDP", "FETP", "PMFP", "TAP"]  # All target variables
# num_models = 100
# all_results = pd.DataFrame()

# for target in target_columns:
#     print(f"Processing target: {target}")

#     # Read and process dataset
#     graphs, labels = read_and_process_dataset(file_path, [target])
#     scaler = StandardScaler()
#     labels_scaled = scaler.fit_transform(labels)

#     # Convert graphs to PyG data objects
#     pyg_graphs = [
#         Data(
#             x=torch.tensor(graph['atom_features'], dtype=torch.float),
#             edge_index=torch.tensor(np.nonzero(graph['adjacency_matrix'])).type(torch.long),
#             y=torch.tensor([labels_scaled[i]], dtype=torch.float)
#         )
#         for i, graph in enumerate(graphs)
#     ]

#     val_metrics = {'mae': [], 'mse': [], 'r2': []}
#     test_metrics = {'mae': [], 'mse': [], 'r2': []}

#     for i in range(num_models):
#         # Split and create DataLoaders
#         X_train, X_temp, y_train, y_temp = train_test_split(pyg_graphs, labels_scaled, train_size=0.8, random_state=i)
#         X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, train_size=0.5, random_state=i)
#         train_loader = DataLoader(X_train, batch_size=8, shuffle=True)
#         val_loader = DataLoader(X_val, batch_size=8, shuffle=False)
#         test_loader = DataLoader(X_test, batch_size=8, shuffle=False)

#         # Train the model
#         model = GCN(num_node_features)
#         optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#         loss_func = torch.nn.MSELoss()
#         scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.8, patience=3, verbose=True)
#         epochs = 300
#         early_stopping_patience = 25
#         trained_model = train(model, train_loader, val_loader, optimizer, loss_func, scheduler, epochs, early_stopping_patience)

#         # Evaluate the model
#         val_mae, val_mse, val_r2 = evaluate_model(trained_model, val_loader)
#         test_mae, test_mse, test_r2 = evaluate_model(trained_model, test_loader)

#         # Save the metrics
#         val_metrics['mae'].append(val_mae)
#         val_metrics['mse'].append(val_mse)
#         val_metrics['r2'].append(val_r2)
#         test_metrics['mae'].append(test_mae)
#         test_metrics['mse'].append(test_mse)
#         test_metrics['r2'].append(test_r2)

#     # Calculate and save the average metrics
#     avg_val_metrics = {metric: np.mean(values) for metric, values in val_metrics.items()}
#     avg_test_metrics = {metric: np.mean(values) for metric, values in test_metrics.items()}

#     result = {
#         'Metric': target,
#         'Validation MAE': avg_val_metrics['mae'],
#         'Validation MSE': avg_val_metrics['mse'],
#         'Validation R2': avg_val_metrics['r2'],
#         'Test MAE': avg_test_metrics['mae'],
#         'Test MSE': avg_test_metrics['mse'],
#         'Test R2': avg_test_metrics['r2']
#     }
#     all_results = all_results.append(result, ignore_index=True)

# # Save the results to a CSV file
# csv_file_path = '/content/all_metrics_gcn_ensemble_results.csv'
# all_results.to_csv(csv_file_path, index=False)

# print(f'Results for all metrics have been saved to {csv_file_path}')
